In [ ]:
import datatree as xrtree
import xarray as xr
import zarr

In [ ]:
core_profiles = xr.open_dataset(
    "~/core_profiles.h5",
    engine="h5netcdf",
    group="core_profiles",
    phony_dims="access",
)

In [ ]:
core_profiles = (
    core_profiles.drop_dims("phony_dim_1")
    .swap_dims(
        {
            "phony_dim_0": "time",
            "phony_dim_2": "rho_tor_norm",
            "phony_dim_3": "ion",
        }
    )
    .set_coords(
        [
            "profiles_1d[]&grid&rho_tor_norm",
            "profiles_1d[]&ion[]&label",
        ]
    )
    .rename_vars(
        {
            "profiles_1d[]&grid&rho_tor_norm": "rho_tor_norm",
            "profiles_1d[]&ion[]&label": "ion",
        }
    )
)

In [ ]:
core_profiles["profiles_1d[]&ion[]&temperature"].isel(time=0, ion=0).plot()

Now to restructure into a zarr file.

In [56]:
with zarr.open("data/core_profiles.zarr", mode="w") as root:
    root.create_dataset("time", data=core_profiles.time.values)
    root.create_dataset("rho_tor_norm", data=core_profiles.rho_tor_norm.values)
    root.create_dataset(
        "electron_temperature",
        data=core_profiles["profiles_1d[]&electrons&temperature"].values,
    )
    # root.attrs["_ARRAY_DIMESIONS"] = ["time", "rho_tor_norm"]
    root["time"].attrs["_ARRAY_DIMENSIONS"] = ["time"]
    root["rho_tor_norm"].attrs["_ARRAY_DIMENSIONS"] = ["time", "rho_tor_norm"]
    root["electron_temperature"].attrs["_ARRAY_DIMENSIONS"] = ["time", "rho_tor_norm"]
    root.create_group("ions")
    zarr.consolidate_metadata("data/core_profiles.zarr")

In [57]:
xr.open_zarr("data/core_profiles.zarr")

<xarray.Dataset> Size: 3kB
Dimensions:               (time: 1, rho_tor_norm: 200)
Coordinates:
    rho_tor_norm          (time, rho_tor_norm) float64 2kB dask.array<chunksize=(1, 200), meta=np.ndarray>
  * time                  (time) float64 8B 0.3003
Data variables:
    electron_temperature  (time, rho_tor_norm) float64 2kB dask.array<chunksize=(1, 200), meta=np.ndarray>